# VacationPy
----
#### Author: Felipe Murillo
#### Date: May 2, 2020
#### Description: Use weather data from 500+ cities across the world to plan the perfect vacation
#### Inputs:  ./WeatherPy/output_data/cities.csv
----

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import sys

# Import API key
sys.path.append('../')
from WeatherPy.config import gkey


### Input City Weather DataFrame


In [2]:
#Read in city weather data
city_weather_df = pd.read_csv("../WeatherPy/output_data/cities.csv",index_col=0)
city_weather_df.head()

,City,City Name,Country,Lng,Lat,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed (mph)
0,hobart,Hobart,AU,147.33,-42.88,51.80,57,20,9.17
1,bengkulu,Bengkulu,ID,102.27,-3.80,78.85,85,100,3.42
2,dudinka,Dudinka,RU,86.18,69.41,30.20,80,75,15.66
3,norman wells,Norman Wells,CA,-126.83,65.28,48.20,45,5,18.34
4,thompson,Thompson,CA,-97.86,55.74,37.40,40,75,13.87


In [3]:
# Store latitude and longitude in locations
city_locations = city_weather_df[["Lat","Lng"]]
city_locations.head()

,Lat,Lng
0,-42.88,147.33
1,-3.80,102.27
2,69.41,86.18
3,65.28,-126.83
4,55.74,-97.86


### Humidity Heatmap

In [4]:
#Configure gmaps
gmaps.configure(api_key=gkey)

In [21]:
# Plot humidity heatmap
fig = gmaps.figure(map_type="HYBRID",
                   center=(0, 0), 
                   zoom_level=1
                  )
heatmap_layer = gmaps.heatmap_layer(
    city_locations,
    weights = city_weather_df["Humidity (%)"],
    max_intensity = city_weather_df["Humidity (%)"].max(),
    point_radius = 10
)
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria

In [6]:
# Create a filter for the following conditions
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness
max_ideal_temp = 80 # deg F
min_ideal_temp = 70 # deg F
max_ideal_wind = 10 # mph
max_ideal_cloud = 0 # cloudiness (%)

perfect_weather = (city_weather_df["Max Temp (F)"] > min_ideal_temp) & \
                  (city_weather_df["Max Temp (F)"] < max_ideal_temp) & \
                  (city_weather_df["Wind Speed (mph)"] < max_ideal_wind) & \
                  (city_weather_df["Cloudiness (%)"] <= max_ideal_cloud)
ideal_cities = city_weather_df.loc[perfect_weather].reset_index()
ideal_cities

,index,City,City Name,Country,Lng,Lat,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed (mph)
0,126,koumac,Koumac,NC,164.28,-20.57,77.86,59,0,4.68
1,183,srivardhan,Srīvardhan,IN,73.02,18.03,77.59,86,0,4.65
2,203,moerai,Moerai,PF,-151.33,-22.43,77.32,70,0,4.52
3,234,hervey bay,Hervey Bay,AU,152.85,-25.30,75.20,60,0,5.82
4,241,xingtai,Xingtai,CN,114.49,37.06,72.39,31,0,2.66
5,268,walvis bay,Walvis Bay,NaN,14.51,-22.96,71.47,39,0,1.16
6,354,chapada dos guimaraes,Chapada dos Guimarães,BR,-55.75,-15.46,75.20,73,0,4.70
7,401,humaita,Humaitá,BR,-63.02,-7.51,76.05,86,0,1.16
8,446,poum,Poum,NC,164.02,-20.23,76.64,65,0,5.91
9,572,henties bay,Henties Bay,NaN,14.28,-22.12,71.13,44,0,8.14


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = ideal_cities[["City Name","Country","Lat","Lng"]]
hotel_df["Hotel Name"] = ""
hotel_df["Hotel Lat"]= ""
hotel_df["Hotel Lng"]= ""
hotel_df

/Users/felipe/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/felipe/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/felipe/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try us

,City Name,Country,Lat,Lng,Hotel Name,Hotel Lat,Hotel Lng
0,Koumac,NC,-20.57,164.28,,,
1,Srīvardhan,IN,18.03,73.02,,,
2,Moerai,PF,-22.43,-151.33,,,
3,Hervey Bay,AU,-25.30,152.85,,,
4,Xingtai,CN,37.06,114.49,,,
5,Walvis Bay,NaN,-22.96,14.51,,,
6,Chapada dos Guimarães,BR,-15.46,-55.75,,,
7,Humaitá,BR,-7.51,-63.02,,,
8,Poum,NC,-20.23,164.02,,,
9,Henties Bay,NaN,-22.12,14.28,,,


In [8]:
# Base URL
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():

    # assemble url and make API request
    print(f"Retrieving Results for Index: {index}, hotel in {row['City Name']}")
    
    # geocoordinates
    target_coordinates = f"{row['Lat']}, {row['Lng']}"
    target_search = "Hotel"
    target_radius = 50000
    target_type = "hotel"
    
    # set up a parameters dictionary
    params = {
        'location': target_coordinates,
        'keyword': target_search,
        'radius': target_radius,
        'type': target_type,
        'key': gkey
        }
    
    # Pull data from API
    hotel_data = requests.get(base_url, params=params).json()
    
    # extract results
    results = hotel_data['results']
    
    try:
        hotel_df.loc[index, 'Hotel Name'] = results[0]["name"]
        hotel_df.loc[index, 'Hotel Lat'] = results[0]["geometry"]["location"]["lat"]
        hotel_df.loc[index, 'Hotel Lng'] = results[0]["geometry"]["location"]["lng"]
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index: 0, hotel in Koumac


/Users/felipe/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


------------
Retrieving Results for Index: 1, hotel in Srīvardhan
------------
Retrieving Results for Index: 2, hotel in Moerai
------------
Retrieving Results for Index: 3, hotel in Hervey Bay
------------
Retrieving Results for Index: 4, hotel in Xingtai
------------
Retrieving Results for Index: 5, hotel in Walvis Bay
------------
Retrieving Results for Index: 6, hotel in Chapada dos Guimarães
------------
Retrieving Results for Index: 7, hotel in Humaitá
------------
Retrieving Results for Index: 8, hotel in Poum
------------
Retrieving Results for Index: 9, hotel in Henties Bay
------------


In [9]:
# Show list of hotels in ideal cities and do a little clean up
hotel_df = hotel_df.rename(columns ={"Lat":"City Lat","Lng":"City Lng"})
hotel_df

,City Name,Country,City Lat,City Lng,Hotel Name,Hotel Lat,Hotel Lng
0,Koumac,NC,-20.57,164.28,Hotel Karem Bay,-20.5478,164.274
1,Srīvardhan,IN,18.03,73.02,The Fern Samali Resort,17.7397,73.1659
2,Moerai,PF,-22.43,-151.33,Le Manotel,-22.4588,-151.323
3,Hervey Bay,AU,-25.30,152.85,Kingfisher Bay Resort,-25.3925,153.03
4,Xingtai,CN,37.06,114.49,Holiday Inn Express Handan East,36.6287,114.537
5,Walvis Bay,NaN,-22.96,14.51,Strand Hotel Swakopmund,-22.6755,14.5219
6,Chapada dos Guimarães,BR,-15.46,-55.75,Hotel Tapajós,-15.5929,-56.0896
7,Humaitá,BR,-7.51,-63.02,Humaitá Quality Hotel,-7.50842,-63.0273
8,Poum,NC,-20.23,164.02,Hotel Karem Bay,-20.5478,164.274
9,Henties Bay,NaN,-22.12,14.28,Cape Cross Lodge,-21.7556,13.9707


In [22]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City Name}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# Create hotel locations
hotel_locations = [hotel_df.loc[hotel][['Hotel Lat','Hotel Lng']] for hotel in hotel_df.index]

# Create marker layer
marker_layer = gmaps.marker_layer(hotel_locations,info_box_content = hotel_info)

# Add marker layer ontop of heat map
fig.add_layer(marker_layer)

#Display Map
fig

Figure(layout=FigureLayout(height='420px'))